In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA


## Gets the dataset from kaggle and put it inside a temprorary location
https://www.kaggle.com/datasets/hiimanshuagarwal/predictive-maintenance-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hiimanshuagarwal/predictive-maintenance-dataset")+ "/predictive_maintenance_dataset.csv"
print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv(path)

df.head()

No missing values to worry about

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df['failure'].value_counts()


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date')['failure'].resample('D').sum().plot()
plt.title("Failures Over Time")
plt.show()

In [ ]:
corr_matrix = df[['failure', 'metric1', 'metric2', 'metric3', 'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9']].corr()
plt.figure(figsize=(8, 8))

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, annot=True, fmt='.2f', cbar_kws={"shrink": .5})

plt.title('Correlation Matrix for Predictive Maintenance Features', fontsize=16, pad=20)
plt.tight_layout()
#plt.savefig('correlation_matrix.png')
plt.show()


In [ ]:
print(df["metric7"]== df["metric8"])

metric7 and metric 8 might be redundant, metric 8/7 has the strongest correlation also metric 2 and metric 4  might help.
it might be a good idea to remove metrics with low correlation from the dataset before creating the models.


A p-value represents the probability of observing a correlation as strong as the one in your sample (or stronger) if there was actually no correlation in the overall population (i.e., if the null hypothesis is true).
How to interpret p-values:

p < 0.05: Traditionally considered statistically significant. This means there's less than a 5% chance you would observe this correlation if no true relationship exists.
p < 0.01: Strongly significant
p < 0.001: Very strongly significant

t = r * sqrt((n-2)/(1-r²))
- r is the correlation coefficient
- n is the sample size (124494)

In [ ]:
import scipy.stats as stats

n = 124494
r = 0.12

# Calculate t-statistic
t = r * math.sqrt((n-2)/(1-r**2))

# Calculate two-tailed p-value
p = 2 * (1 - stats.t.cdf(abs(t), df=n-2))

print(f"t-statistic: {t}")
print(f"p-value: {p}")

the p value is so small that python rounds it to zero.
this mean even the metrics with smallest correlation is statistically significant.